# SK ChatBot

Learning objectives:
- Difference between LLM and GPT models
- System vs user messages
- Working keeping a conversation
- Advantages of using SK:
  - HttpClient handling and built-in retry logic
  - Pre-defined models (Prompt & Completion)
  - Process prompts as pipes
  - Prompts and prompt templates (semantic functions)
  - Process native functions as semantic functions


## Load the required .NET packages and supporting classes

In [34]:
#r "nuget: Microsoft.SemanticKernel, 1.3.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.3.0"
#r "nuget: dotenv.net"

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;


Installed Packages dotenv.net, 3.1.3 Microsoft.SemanticKernel, 1.3.0 Microsoft.SemanticKernel.Core, 1.3.0

## Load the OpenAI variables from environment variables or an .env file

In [35]:
DotEnv.Load();
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
Console.WriteLine($"Using deployment: {deploymentName} at {endpoint} with key {apiKey.Substring(0, 5)}...");

Using deployment: gpt at https://alemorfr.openai.azure.com/ with key 32c87...


### Creat a chat completion service

- Registration: `WithAzureChatCompletionServices` registers an ICompletion service with the kernel
- Retries: Kernel API calls come with default retry logic for timeouts and throttling.

In [36]:
AzureOpenAIChatCompletionService chatCompletionService = new(deploymentName, endpoint, apiKey);

### Add the first message into history

In [37]:
var chatHistory = new ChatHistory("You are a general assistant.");

### Print the last message in history

In [38]:
private Task MessageOutputAsync(ChatHistory chatHistory)
{
    var message = chatHistory.Last();

    Console.WriteLine($"{message.Role}: {message.Content}");
    Console.WriteLine("------------------------");

    return Task.CompletedTask;
}

### Add the first message and completion

In [39]:
// First user message
chatHistory.AddUserMessage("Hi, I'm looking for book suggestions");
await MessageOutputAsync(chatHistory);

var reply = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
chatHistory.Add(reply);
await MessageOutputAsync(chatHistory);

user: Hi, I'm looking for book suggestions
------------------------
assistant: Of course! What kind of books are you interested in? Fiction, non-fiction, mystery, romance, science fiction, etc? Any specific genres or themes you enjoy?
------------------------


## Enter the while loop and process the user prompts and system completions

In [40]:
while (true) {
    var userMessage = await InteractiveKernel.GetInputAsync("Your message");
    if (userMessage == "exit") {
        break;
    }
    chatHistory.AddUserMessage(userMessage);
    await MessageOutputAsync(chatHistory);

    var reply = await chatCompletionService.GetChatMessageContentAsync(chatHistory);
    chatHistory.Add(reply);
    await MessageOutputAsync(chatHistory);
}

user: What is the speed of light?
------------------------
assistant: The speed of light is approximately 299,792,458 meters per second or about 186,282 miles per second. It is a fundamental constant of nature and the maximum speed at which all matter and information can travel through space.
------------------------
